## Amazon AgentCore Bedrock Code Interpreter チュートリアル

このチュートリアルでは、AgentCore Bedrock Code Interpreter を使用して以下を行う方法を説明します：
1. サンドボックス環境のセットアップ
2. データの読み込みと分析
3. サンドボックス環境でのコード実行
4. 結果の処理と取得

## 前提条件
- Bedrock AgentCore Code Interpreter へのアクセス権を持つ AWS アカウント
- Code Interpreter リソースを作成・管理するために必要な IAM 権限
- 必要な Python パッケージのインストール（boto3 および bedrock-agentcore を含む）
- サンプルデータファイル (data.csv)
- 分析スクリプト (stats.py)


## IAM 実行ロールには以下の IAM ポリシーをアタッチする必要があります

~~~ {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:CreateCodeInterpreter",
                "bedrock-agentcore:StartCodeInterpreterSession",
                "bedrock-agentcore:InvokeCodeInterpreter",
                "bedrock-agentcore:StopCodeInterpreterSession",
                "bedrock-agentcore:DeleteCodeInterpreter",
                "bedrock-agentcore:ListCodeInterpreters",
                "bedrock-agentcore:GetCodeInterpreter"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "arn:aws:logs:*:*:log-group:/aws/bedrock-agentcore/code-interpreter*"
        }
    ]
}



## 動作の仕組み

コード実行サンドボックスは、Code Interpreter、シェル、ファイルシステムを備えた隔離環境を作成することで、エージェントがユーザーのクエリを安全に処理できるようにします。Large Language Model がツール選択を支援した後、このセッション内でコードが実行され、その後ユーザーまたはエージェントに返されて統合されます。

![architecture local](code-interpreter.png)

## 1. 環境のセットアップ

まず、必要なライブラリをインポートし、Code Interpreter クライアントを初期化します。

In [ ]:
!pip install --upgrade -r requirements.txt

In [ ]:
from bedrock_agentcore.tools.code_interpreter_client import CodeInterpreter
import json
from typing import Dict, Any, List

# Initialize the Code Interpreter with your AWS region
code_client = CodeInterpreter('us-west-2')
code_client.start()

## 2. ローカルファイルの読み込み

サンプルデータファイルと分析スクリプトの内容を読み込みます。

In [ ]:
def read_file(file_path: str) -> str:
    """Helper function to read file content with error handling"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"エラー: ファイル '{file_path}' が見つかりませんでした。")
        return ""
    except Exception as e:
        print(f"エラーが発生しました: {e}")
        return ""

# Read both files
data_file_content = read_file("samples/data.csv")
code_file_content = read_file("samples/stats.py")

## 3. サンドボックス環境用ファイルの準備

サンドボックス環境に作成するファイルを定義する構造を作成します。

In [ ]:
files_to_create = [
    {
        "path": "data.csv",
        "text": data_file_content
    },
    {
        "path": "stats.py",
        "text": code_file_content
    }
]

## 4. ツール呼び出し用ヘルパー関数の作成

このヘルパー関数は、サンドボックスツールの呼び出しとレスポンスの処理を容易にします。アクティブなセッション内では、サポートされている言語（Python、JavaScript）でコードを実行し、依存関係の設定に基づいてライブラリにアクセスし、可視化を生成し、実行間で状態を維持できます。

In [ ]:
def call_tool(tool_name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
    """Helper function to invoke sandbox tools
    
    Args:
        tool_name (str): Name of the tool to invoke
        arguments (Dict[str, Any]): Arguments to pass to the tool
        
    Returns:
        Dict[str, Any]: JSON formatted result
    """
    response = code_client.invoke(tool_name, arguments)
    for event in response["stream"]:
        return json.dumps(event["result"])

## 5. サンドボックスへのファイル書き込み

ファイルをサンドボックス環境に書き込み、正常に作成されたことを確認します。

In [ ]:
# Write files to sandbox
writing_files = call_tool("writeFiles", {"content": files_to_create})
print("ファイル書き込み結果:")
print(writing_files)

# Verify files were created
listing_files = call_tool("listFiles", {"path": ""})
print("\nサンドボックス内のファイル:")
print(listing_files)

## 6. 分析の実行

サンドボックス環境で分析スクリプトを実行し、結果を処理します。

In [ ]:
import pprint

# Execute the analysis script
code_execute_result = call_tool("executeCode", {
    "code": files_to_create[1]['text'],
    "language": "python",
    "clearContext": True
})

# Parse and display results
analysis_results = json.loads(code_execute_result)
print("完全な分析結果:")
pprint.pprint(analysis_results)

print("\n分析からの標準出力:")
print(analysis_results['structuredContent']['stdout'])

## 7. クリーンアップ

最後に、Code Interpreter セッションを停止してクリーンアップします。セッションの使用が完了したら、リソースを解放し不要な課金を避けるためにセッションを停止する必要があります。

In [ ]:
# Stop the Code Interpreter session
code_client.stop()
print("Code Interpreter セッションが正常に停止しました！")

## まとめ

このチュートリアルでは、以下について学びました：
- Code Interpreter セッションの初期化
- 分析用ファイルの読み込みと準備
- サンドボックス環境でのコード実行
- 結果の処理と表示
- リソースのクリーンアップ